# README.md

### For each pt with an ADT ending date, find its T date and T value around 6months later
#rules: if has a T test/value within the 6 months (180-210 days), take the closet to 180 days; if not, take one from within 5-6 months (150-180 days), also take the closet to 180 days  

## Table of Contents
* [Read data,clean up and get stats](#first-bullet)
* [Get the closest date to 6 months](#second-bullet)
* [Output results](#third-bullet)

# Read data, clean up and get stats <a class="anchor" id="first-bullet"></a>

In [38]:
import numpy as np
import pandas as pd
import sys, getopt
import csv
from datetime import datetime
from datetime import timedelta, date
import itertools

from matplotlib import pyplot
from pylab import *
import matplotlib.pyplot as plt

%matplotlib inline  
import seaborn as sns

In [39]:
df1=pd.read_excel("./data/T_recovery_localized_disease_167pts_cohort.xlsx", header=0)
df1.fillna('', inplace=True)
#df1['inital_ADT_end_date'] = pd.to_datetime(df1['inital_ADT_end_date'])

df2=pd.read_excel("./data/pca_tresults_20210119_BP_fixed.xlsx", header=0)
#df3=pd.read_csv("just_start_empty.csv", header=0)

In [40]:
df1[:3]

,person_id,status_manual_ok,status_tbaseline,status_rec,ADT_start,ADT_end,baseline_tdate,baseline_tvalue,rec_tdate,rec_tvalue,person_id.1,Salvage RT+ADT (Y/N),evidence for E,ADT used as part of localized disease (Y/N)
0,38,Y,Y,Y,2015-11-18,2016-07-18,2015-11-18,288.37,2018-04-16 00:00:00,317.16,38,N,"Given his age and high risk disease, he is not...",Y
1,81,Y,Y,Y,2014-04-15,2016-06-09,2014-04-10,313.07,2017-04-03 00:00:00,309.59,81,Y,,Y
2,119,Y,Y,Y,2013-07-02,2013-10-02,2013-06-17,403.10,2015-09-21 00:00:00,331.74,119,N,,Y


In [4]:
#df3=pd.read_csv("just_start_empty.csv", header=0)
df2.replace(r'', np.NaN)
df2[:3]

,person_id,cancer_type_id,year_of_diagnosis,month_of_diagnosis,day_of_diagnosis,dx_date,result_date,loinc_code,source_test_name,test_results
0,2,2,2003,1.0,7.0,2003-01-07,2006-09-05,49041-7,TESTOSTERONE,351.0
1,2,2,2003,1.0,7.0,2003-01-07,2007-01-09,49041-7,TESTOSTERONE,304.0
2,2,2,2003,1.0,7.0,2003-01-07,2007-05-17,49041-7,TESTOSTERONE,259.0


In [5]:
df2[:1]

,person_id,cancer_type_id,year_of_diagnosis,month_of_diagnosis,day_of_diagnosis,dx_date,result_date,loinc_code,source_test_name,test_results
0,2,2,2003,1.0,7.0,2003-01-07,2006-09-05,49041-7,TESTOSTERONE,351.0


In [6]:
list_pts_adt_table=df1["person_id"].to_list()
list_pts_tvalue_table=df2["person_id"].to_list()

In [7]:
print("pt number in adt table is:")
print(len(list_pts_adt_table))
print("")
print("pt number in t value table is:")
print(len(list_pts_tvalue_table))

pt number in adt table is:
167

pt number in t value table is:
18643


In [8]:
index_list=df2.columns
index_list

Index(['person_id', 'cancer_type_id', 'year_of_diagnosis',
       'month_of_diagnosis', 'day_of_diagnosis', 'dx_date', 'result_date',
       'loinc_code', 'source_test_name', 'test_results'],
      dtype='object')

In [9]:
print(df2.shape)
#delete some useless columns
del_col=['cancer_type_id', 'year_of_diagnosis', 'month_of_diagnosis', 'day_of_diagnosis', 'dx_date','loinc_code', 'source_test_name']

for col in del_col:
    del df2[col]
print(df2.shape)

(18643, 10)
(18643, 3)


In [19]:
y=81

df1_pt=df1[(df1["person_id"] == y)]
df2_pt=df2[(df2["person_id"] == y)]

In [20]:
df1_pt[:2]

,person_id,status_manual_ok,status_tbaseline,status_rec,ADT_start,ADT_end,baseline_tdate,baseline_tvalue,rec_tdate,rec_tvalue,person_id.1,Salvage RT+ADT (Y/N),evidence for E,ADT used as part of localized disease (Y/N)
1,81,Y,Y,Y,2014-04-15,2016-06-09,2014-04-10,313.07,2017-04-03 00:00:00,309.59,81,Y,,Y


In [21]:
df2_pt

,person_id,result_date,test_results
274,81,2014-04-10,313.07
275,81,2014-05-15,10.00
276,81,2016-09-27,14.52
277,81,2016-12-29,228.03
278,81,2017-04-03,309.59
279,81,2017-07-05,300.13
280,81,2017-10-16,215.23
281,81,2018-04-16,223.67
282,81,2018-11-29,269.96
283,81,2019-05-06,208.60


In [22]:
tvalue=df2_pt.loc[df2_pt['result_date'] == ('2017-02-01 00:00:00')]

In [23]:
test=df2_pt[:1].test_results.values[0]
test

313.07

In [ ]:
# Get the closest date to 6 months  <a class="anchor" id="second-bullet"></a>

In [24]:
def closest(lst, K): 
      return lst[min(range(len(lst)), key = lambda i: abs(lst[i] - K))] 

In [29]:
result_list=[]

for y in list_pts_adt_table:
    
    df1_pt=df1[(df1["person_id"] == y)]
    a=df1_pt['ADT_end'].to_list()[0]
    #add 6 months to the inital ADT end date
    a_plus_5mons=a + timedelta(days=150)
    a_plus_6mons=a + timedelta(days=180)
    a_plus_7mons=a + timedelta(days=210)
    #pick up initial ADT window 
    
    df2_pt=df2[(df2["person_id"] == y)]    
    each_pt_tdates = df2_pt['result_date'].to_list()

    ##get rid of nan
    cleaned_a=[]
    cleaned_a= [x for x in each_pt_tdates if str(x) != 'NaN']
    cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']
    cleaned_aaa= [x for x in cleaned_aa if str(x) != 'NaT']
    cleaned_b1= [x for x in cleaned_aaa if x>=a_plus_6mons and x <=a_plus_7mons]
    cleaned_b2= [x for x in cleaned_aaa if x>=a_plus_5mons and x <=a_plus_6mons]

    start_sorted_bin1=sorted(cleaned_b1)
    start_sorted_bin2=sorted(cleaned_b2)    

    if(len(start_sorted_bin1)>0):
        #apply the closet function
        tdate=closest(start_sorted_bin1,a_plus_6mons)
        #pick up T-value for T-date
        df2_pt_pick=df2_pt[(df2_pt['result_date']==tdate)]
        tvalue=df2_pt_pick.test_results.values[0]
        status='btw 180-210days'
        
    if(len(start_sorted_bin1)==0):
        if(len(start_sorted_bin2)==0):            
            tdate='NaN'
            status='NaN'
            tvalue='NaN'  
            
        if(len(start_sorted_bin2)>0):                          
            #apply the closet function
            tdate=closest(start_sorted_bin2,a_plus_6mons)          
            #pick up T-value for T-date
            df2_pt_pick=df2_pt[(df2_pt['result_date']==tdate)]
            if len(df2_pt_pick.test_results.values)>0:
                tvalue=df2_pt_pick.test_results.values[0]
            if len(df2_pt_pick.test_results.values)==0:
                start_sorted_bin2.remove(tdate)
                tdate=closest(start_sorted_bin2,a_plus_6mons)
                #pick up T-value for T-date
                df2_pt_pick=df2_pt[(df2_pt['result_date']==tdate)]                
            status='btw 150-180days'
            
    print(y, a,  a_plus_6mons, tdate, tvalue,status)
    result_per_pt=[y, a, a_plus_6mons, tdate, tvalue,status]
    result_list.append(result_per_pt)

38 2016-07-18 00:00:00 2017-01-14 00:00:00 2017-01-20 00:00:00 18.85 btw 180-210days
81 2016-06-09 00:00:00 2016-12-06 00:00:00 2016-12-29 00:00:00 228.03 btw 180-210days
119 2013-10-02 00:00:00 2014-03-31 00:00:00 NaN NaN NaN
870 2017-10-13 00:00:00 2018-04-11 00:00:00 2018-04-10 00:00:00 228.0 btw 150-180days
1257 2015-07-08 00:00:00 2016-01-04 00:00:00 2016-01-27 00:00:00 203.25 btw 180-210days
1503 2018-11-26 00:00:00 2019-05-25 00:00:00 NaN NaN NaN
1508 2017-04-16 00:00:00 2017-10-13 00:00:00 2017-10-21 00:00:00 54.0 btw 180-210days
1583 2013-09-11 00:00:00 2014-03-10 00:00:00 NaN NaN NaN
1704 2015-06-05 00:00:00 2015-12-02 00:00:00 2015-12-11 00:00:00 125.0 btw 180-210days
2061 2019-02-06 00:00:00 2019-08-05 00:00:00 NaN NaN NaN
2063 2018-03-20 00:00:00 2018-09-16 00:00:00 2018-09-20 00:00:00 376.65 btw 180-210days
2200 2014-11-03 00:00:00 2015-05-02 00:00:00 NaN NaN NaN
2623 2019-10-03 00:00:00 2020-03-31 00:00:00 NaN NaN NaN
2652 2019-06-01 00:00:00 2019-11-28 00:00:00 2019-12-

In [32]:
df_results=pd.DataFrame(result_list)
df_results.columns=(['person_id', 'inital_ADT_end_date','6 mons after ADT', 'Tdate closest to 6 mons after ADT', "Tvalue","Tdate_status"])

In [33]:
#create a column for the time gap to screen the ones outside the 5mons to 7 mons window  
df_results['Tvalue_ADT+6mons_gap'] = df_results['Tdate closest to 6 mons after ADT'] - df_results['6 mons after ADT']
#create a column for the inital adt use window as days
df_results['Tvalue_ADT+6mons_gap_days']=df_results['Tvalue_ADT+6mons_gap'].dt.days

In [34]:
df_results

,person_id,inital_ADT_end_date,6 mons after ADT,Tdate closest to 6 mons after ADT,Tvalue,Tdate_status,Tvalue_ADT+6mons_gap,Tvalue_ADT+6mons_gap_days
0,38,2016-07-18,2017-01-14,2017-01-20,18.85,btw 180-210days,6 days,6.0
1,81,2016-06-09,2016-12-06,2016-12-29,228.03,btw 180-210days,23 days,23.0
2,119,2013-10-02,2014-03-31,NaT,NaN,NaN,NaT,NaN
3,870,2017-10-13,2018-04-11,2018-04-10,228,btw 150-180days,-1 days,-1.0
4,1257,2015-07-08,2016-01-04,2016-01-27,203.25,btw 180-210days,23 days,23.0
...,...,...,...,...,...,...,...,...
162,180346,2019-09-18,2020-03-16,NaT,NaN,NaN,NaT,NaN
163,180780,2020-01-15,2020-07-13,NaT,NaN,NaN,NaT,NaN
164,185086,2020-02-07,2020-08-05,NaT,NaN,NaN,NaT,NaN
165,186766,2020-02-03,2020-08-01,NaT,NaN,NaN,NaT,NaN


In [ ]:
# Output results  <a class="anchor" id="third-bullet"></a>

In [36]:
df_results.to_csv("./data/localized_167pts_cohort_Tdate_Tvalue_ADTwindow_closest_to_6mons_after_ADT.csv", encoding='utf-8', index=False)